Integrantes:

1. Ignacio Morande
2. Fernando Contreras
3. Alberto Bella

# Tarea 2

# Parte práctica (3 puntos)
La NASA mantiene la información de varios cometas y quiere determinar alguna manera de predecir el diametro de un cometa. Específicamente, han analizado en forma manual una muestra de 100.000 asteroides. Los datos consisten en 27 variables, con distinta información como por ejemplo nombre del asteroide, su periodo orbital, su periodo de rotación, etc. Todos los datos existentes, se encuentran en un puro archivo llamado asteroidTrain.csv. Mientras que las descripciones de cada una de las variables se encuentran en el archivo tarea2Informacion.txt

Desafortunadamente, la NASA todavía no ha evaluado 37.681 asteroides y no tienen tiempo para realizarlo. Por lo mismo, le piden que aplique una red neuronal feed forward para obtener una predicción de estos asteroides.

1. Lea los datos y borre las variables/asteroides que estime necesario. En caso que crea que la base de datos todavía es demasiado grande para aplicar los modelos, usted puede tomar una muestra de la misma (1.5 punto).
2. Entrene un modelo feed forward. Realice una busqueda de parámetros modifique el número de capas, de neuronas, funciones de activación, epocas, etc. Seleccione un solo modelo de los seleccionados y muestre ese modelo (0.5 puntos).
3. Seleccione uno de los modelos del punto anterior y evalue los 37.681 asteroides que la NASA no ha evaluado. Se deberá generar un archivo csv con 37.681 filas, cada fila deberá ser una estimación (1 punto). 

El punto de evaluación final será una competencia entre todas las tareas basados en los MSE más bajo obtenido por cada grupo. El puntaje final será una regresión lineal entre el peor y mejor puntaje.

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
from plotnine import *
import matplotlib.pyplot as plt

df_test = pd.read_csv('asteroidEval.csv')
df = pd.read_csv('asteroidTrain.csv')
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,full_name,a,e,G,i,om,w,q,ad,per_y,data_arc,condition_code,n_obs_used,H,diameter,extent,albedo,rot_per,GM,BV,UB,IR,spec_B,spec_T,neo,pha,moid
0,97632 (2000 EL145),3.069866,0.184417,NaN,3.620920,308.458533,249.570070,2.503729,3.636003,5.378824,6803.0,0.0,614,14.6,6.611,NaN,0.070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,1.50522
1,49808 (1999 XD40),2.386539,0.104566,NaN,13.003513,105.866881,329.104052,2.136988,2.636089,3.686897,7672.0,0.0,1247,14.7,4.255,NaN,0.170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,1.16197
2,390390 (2013 WC84),3.018175,0.084529,NaN,14.247933,245.449168,115.242451,2.763053,3.273297,5.243544,5904.0,0.0,181,15.6,5.625,NaN,0.024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,1.79743
3,14319 (1978 US5),2.793049,0.290215,NaN,16.677722,17.862425,287.769691,1.982465,3.603633,4.667948,25107.0,0.0,955,14.0,4.224,NaN,0.324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,1.04034
4,321079 (2008 SW97),2.721394,0.098602,NaN,1.975506,271.706257,248.448520,2.453059,2.989728,4.489472,8027.0,0.0,251,16.4,3.525,NaN,0.043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,1.46299


In [ ]:
# G, extent, rot_per, GM, BVm UB, IR, spec_B, spec_T: to many NaN
# full_name: not relevant info
# neo, pha: binary variable
# condition_code: categorical variable
drop_col = ['full_name', 'G', 'extent', 'rot_per', 'GM', 'BV', 'UB', 'IR', 'spec_B', 'spec_T', 'condition_code', 'neo', 'pha']

df_train = df.drop(columns = drop_col)
df_test = df_test.drop(columns = drop_col)
df_train = df_train.dropna() # Elimina filas con NaN
df_test = df_test.dropna()
df_train # Quedan 98837 filas

,a,e,i,om,w,q,ad,per_y,data_arc,n_obs_used,H,diameter,albedo,moid
0,3.069866,0.184417,3.620920,308.458533,249.570070,2.503729,3.636003,5.378824,6803.0,614,14.6,6.611,0.070,1.50522
1,2.386539,0.104566,13.003513,105.866881,329.104052,2.136988,2.636089,3.686897,7672.0,1247,14.7,4.255,0.170,1.16197
2,3.018175,0.084529,14.247933,245.449168,115.242451,2.763053,3.273297,5.243544,5904.0,181,15.6,5.625,0.024,1.79743
3,2.793049,0.290215,16.677722,17.862425,287.769691,1.982465,3.603633,4.667948,25107.0,955,14.0,4.224,0.324,1.04034
4,2.721394,0.098602,1.975506,271.706257,248.448520,2.453059,2.989728,4.489472,8027.0,251,16.4,3.525,0.043,1.46299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,2.758028,0.092627,1.994054,173.649449,216.612757,2.502559,3.013496,4.580428,8978.0,878,14.9,3.356,0.157,1.50731
99996,2.715932,0.239266,16.789523,34.531718,44.097353,2.066102,3.365761,4.475963,6595.0,201,16.1,4.724,0.055,1.11044
99997,3.188786,0.278970,15.177104,188.342499,255.934423,2.299209,4.078363,5.694376,5377.0,141,15.8,4.148,0.059,1.36906
99998,2.754535,0.106028,8.758664,230.483443,34.567890,2.462477,3.046593,4.571730,7973.0,835,15.0,2.749,0.177,1.45246


In [ ]:
from sklearn.model_selection import train_test_split

# outliers

# sampling
X = df_train[['H', 'albedo', 'i', 'w', 'om', 'a', 'q']].astype(np.float32) # 'H', 'albedo', 'i', 'w', 'om', 'a'
y = df_train[['diameter']].astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=42)

# gridsearch
df_train.corr(method='pearson')

,a,e,i,om,w,q,ad,per_y,data_arc,n_obs_used,H,albedo,moid
a,1.000000,0.020140,0.145463,0.000677,-0.003938,0.351115,0.986323,0.937665,-0.020186,-0.050478,-0.132787,-0.112676,0.354295
e,0.020140,1.000000,0.144074,0.000618,0.011340,-0.530544,0.114501,0.046154,-0.025897,-0.075504,0.195677,-0.017901,-0.507438
i,0.145463,0.144074,1.000000,-0.010225,-0.005617,0.090659,0.136505,0.089660,-0.189003,-0.219579,-0.038256,-0.087733,0.130376
om,0.000677,0.000618,-0.010225,1.000000,-0.108038,-0.003120,0.001258,0.001395,0.002191,-0.024764,0.003344,0.000486,-0.003735
w,-0.003938,0.011340,-0.005617,-0.108038,1.000000,-0.009009,-0.002542,-0.001936,-0.003755,0.011688,-0.009793,-0.000882,-0.009146
q,0.351115,-0.530544,0.090659,-0.003120,-0.009009,1.000000,0.191985,0.090924,-0.019968,-0.084227,-0.382581,-0.271085,0.996854
ad,0.986323,0.114501,0.136505,0.001258,-0.002542,0.191985,1.000000,0.966789,-0.017642,-0.038081,-0.071832,-0.070380,0.195871
per_y,0.937665,0.046154,0.089660,0.001395,-0.001936,0.090924,0.966789,1.000000,-0.007804,-0.010632,-0.033245,-0.019190,0.092174
data_arc,-0.020186,-0.025897,-0.189003,0.002191,-0.003755,-0.019968,-0.017642,-0.007804,1.000000,0.750927,-0.672794,0.253907,-0.027549
n_obs_used,-0.050478,-0.075504,-0.219579,-0.024764,0.011688,-0.084227,-0.038081,-0.010632,0.750927,1.000000,-0.783566,0.448320,-0.093925


In [ ]:
from tensorflow.keras import *

#Creating and training a model
inputLayer = layers.Input(shape=(7,))
hiddenLayer = layers.Dense(9, activation='linear',use_bias = True)(inputLayer)
hiddenLayer2 = layers.Dense(7, activation='linear',use_bias = True)(hiddenLayer)
hiddenLayer3 = layers.Dense(5, activation='linear',use_bias = True)(hiddenLayer2)
outputLayer = layers.Dense(1, activation='linear',use_bias = True)(hiddenLayer3)

feedForward = models.Model(inputLayer, outputLayer)
feedForward.compile(loss='MeanSquaredError', optimizer = 'adam', metrics=['RootMeanSquaredError'])
feedForward.fit(X_train[:10000], y_train[:10000], epochs=400, batch_size=8, shuffle=True, verbose=0)

# variar bach-size y epochs

# variar capas y neuronas
resultado = feedForward.evaluate(X_test, y_test, return_dict=True)
resultado

1082/1082 [==============================] - 2s 1ms/step - loss: 41.3677 - root_mean_squared_error: 6.4318


{'loss': 41.367713928222656, 'root_mean_squared_error': 6.431773662567139}

In [ ]:
resultado

{'loss': 41.367713928222656, 'root_mean_squared_error': 6.431773662567139}

In [ ]:
df_test

,a,e,i,om,w,q,ad,per_y,data_arc,n_obs_used,H,albedo,moid
0,3.969733,0.082587,6.263280,159.529531,338.904665,3.641886,4.297579,7.909520,14333.0,2241,11.0,0.058,2.656530
1,2.262880,0.096618,22.340191,52.502912,87.172523,2.044245,2.481515,3.404085,2.0,13,17.1,0.076,1.174060
2,3.072270,0.103533,9.503231,217.293949,191.020252,2.754189,3.390352,5.385145,10318.0,1588,13.0,0.111,1.763990
3,2.766916,0.208954,10.208236,161.123803,188.856431,2.188758,3.345075,4.602590,10363.0,664,15.0,0.044,1.182110
4,2.361788,0.259924,9.486384,100.768260,231.685109,1.747903,2.975672,3.629690,15207.0,803,15.7,0.161,0.753967
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37676,3.948805,0.121336,12.100826,176.921861,253.018391,3.469671,4.427939,7.847055,8811.0,263,14.5,0.044,2.510560
37677,2.985065,0.085091,9.437864,165.845432,49.880238,2.731062,3.239067,5.157495,9062.0,706,14.4,0.111,1.737330
37678,2.250727,0.146572,4.952574,136.066108,218.906512,1.920834,2.580621,3.376701,7823.0,483,16.1,0.242,0.917571
37679,2.692312,0.182102,14.004905,177.379901,92.464686,2.202038,3.182587,4.417702,6147.0,434,15.9,0.088,1.240040


In [ ]:
test = pd.DataFrame(feedForward.predict(df_test[['H', 'albedo', 'i', 'w', 'om', 'a', 'q']])[:, 0])
test

,0
0,25.707397
1,-0.757519
2,15.957624
3,9.553997
4,4.217996
...,...
37202,10.225178
37203,9.036356
37204,-0.518005
37205,4.011283


In [ ]:
test.to_csv('test.csv', index=False, header=False)

# Teoría (3 puntos)
Una de las ventajas al usar redes multicapas es la posiblidad de abordar problemas con múltiples clases a través de la función softmax. 

Imaginemos que tenemos una red feed forward de múltiples capas con $k$ neuronas de salidas, función de activación softmax, y función de perdida $\displaystyle L=-\sum_{i=1}^ky_i\ln(o_i)$ (Note que la función de error esta definido para un punto, no un set de puntos). En este error, $o_i$ es la salida de la neurona de la clase $i$ e $y$ es un vector de $k$ valores correspondiente a la transformación one-hot-encoding de la clase (recuerde que tenemos $k$ clases).<br>
Por ejemplo, si el problema tiene 4 clases y el punto pertenece a la clase 3, entonces $y=[0,0,1,0]$, por lo tanto $y_1=0$, $y_2=0$, $y_3=1$ $y_4=0$.<br>
Note que nunca se define el número de puntos del set de datos, no importa en esta tarea.
 
Demuestre que $\displaystyle \frac{\partial L}{\partial h_i}=\sum_{j=1}^k\frac{\partial L}{\partial o_j}\frac{\partial o_j}{\partial h_i}=o_i-y_i$, donde $\displaystyle o_i=\frac{\exp(h_i)}{\sum_{j=1}^k\exp(h_j)}$ y $h_i$ es el valor continuo correspondiente a la neurona $i$ que es transformado a una probabilidad. <br><br>

Para realizar esta demostración realize lo siguiente
1. Demuestre que $\displaystyle\frac{\partial o_i}{\partial h_j}=-o_io_j$ si $i\neq j$ (1 punto)<br>
2. Demuestre que $\displaystyle\frac{\partial o_i}{\partial h_j}=o_i(1-o_i)$ si $i=j$ (1 punto)<br>
3. Utilizando las demostraciones anteriores demuestre $\displaystyle \frac{\partial L}{\partial h_i}=\sum_{j=1}^k\frac{\partial L}{\partial o_j}\frac{\partial o_j}{\partial h_i}=o_i-y_i$ (1 punto)

En caso que no pueda demostrar los puntos 1 y 2 acepte esas relaciones y demuestre directamente el punto 3 por un solo punto.



---



1. Demuestre que $\displaystyle \frac{\partial o_i}{\partial h_j}=-o_io_j$ si $i\neq j$: <br>

Sabemos que la funcion de activiación es la función softmax

$\displaystyle o_i = \frac{\exp(h_i)}{\sum_{j=1}^k\exp(h_j)}$

$\displaystyle \frac{\partial o_i}{\partial h_j} = \frac{\exp(h_i)\cdot \frac{\partial h_i}{\partial h_j} \cdot \sum_{j=1}^{k}{\exp(h_j)} - \exp(h_i) \cdot \frac{\partial}{\partial h_j} \sum_{j=1}^{k}{\exp(h_j)}}{[\sum_{j=1}^{k}{\exp(h_j)}]^2}$

$\displaystyle \frac{\partial o_i}{\partial h_j} = \frac{0 - \exp(h_i) \cdot \exp(h_j)}{[\sum_{j=1}^{k}{\exp(h_j)}]^2} \text{, si } i \neq j$

$\displaystyle \frac{\partial o_i}{\partial h_j} = \frac{- \exp(h_i) \cdot \exp(h_j)}{\sum_{j=1}^{k}{\exp(h_j)} \cdot \sum_{j=1}^{k}{\exp(h_j)}}$

$\displaystyle \frac{\partial o_i}{\partial h_j} = -o_io_j$

2. Demuestre que $\displaystyle\frac{\partial o_i}{\partial h_j}=o_i(1-o_i)$ si $i=j$: <br>

$\displaystyle \frac{\partial o_i}{\partial h_j} = \frac{\exp(h_i)\cdot \frac{\partial h_i}{\partial h_j} \cdot \sum_{j=1}^{k}{\exp(h_j)} - \exp(h_i) \cdot \frac{\partial}{\partial h_j} \sum_{j=1}^{k}{\exp(h_j)}}{[\sum_{j=1}^{k}{\exp(h_j)}]^2}$

$\displaystyle \frac{\partial o_i}{\partial h_j} = \frac{\exp(h_i)\cdot \sum_{j=1}^{k}{\exp(h_j)} - \exp(h_i) \cdot \exp(h_i)}{[\sum_{j=1}^{k}{\exp(h_j)}]^2} \text{, si } i = j$

$\displaystyle \frac{\partial o_i}{\partial h_j} =\frac{\exp(h_i)\cdot \sum_{j=1}^{k}{\exp(h_j)}}{[\sum_{j=1}^{k}{\exp(h_j)}]^2} - \frac{\exp(h_i)^2}{[\sum_{j=1}^{k}{\exp(h_j)}]^2}$

$\displaystyle \frac{\partial o_i}{\partial h_j} =\frac{\exp(h_i)}{\sum_{j=1}^{k}{\exp(h_j)}} - (\frac{\exp(h_i)}{\sum_{j=1}^{k}{\exp(h_j)}})^2$

$\displaystyle \frac{\partial o_i}{\partial h_j} = o_i - o_i^2$

$\displaystyle \frac{\partial o_i}{\partial h_j} =o_i(1 - o_i)$

3. Demuestre $\displaystyle \frac{\partial L}{\partial h_i}=\sum_{j=1}^k\frac{\partial L}{\partial o_j}\frac{\partial o_j}{\partial h_i}=o_i-y_i$: <br>

Sabemos que la funcion de error es la siguiente:

$\displaystyle L=-\sum_{i=1}^ky_i\ln(o_i)$

$\displaystyle \frac{\partial L}{\partial h_i} = - \frac{\partial}{\partial h_i} \sum_{i=1}^ky_i\ln(o_i)$

$\displaystyle \frac{\partial L}{\partial h_i} = - \frac{\partial}{\partial h_i} \cdot [y_1 \ln(o_1) + y_2 \ln(o_2) + ... + y_i \ln(o_i) + ... + y_k \ln(o_k)] $

$\displaystyle \frac{\partial L}{\partial h_i} = - [\frac{\partial}{\partial h_i} y_1 \ln(o_1) + \frac{\partial}{\partial h_i} y_2 \ln(o_2) + ... + \frac{\partial}{\partial h_i} y_i \ln(o_i) + ... + \frac{\partial}{\partial h_i} y_k \ln(o_k)]$

$\displaystyle \frac{\partial L}{\partial h_i} = - [\frac{y_1}{o_1} \frac{\partial o_1}{\partial h_i} + \frac{y_2}{o_2} \frac{\partial o_2}{\partial h_i} + ... + \frac{y_i}{o_i} \frac{\partial o_i}{\partial h_i} + ... + \frac{y_k}{o_k} \frac{\partial o_k}{\partial h_i}]$

$\displaystyle \frac{\partial L}{\partial h_i} = - [\frac{y_1}{o_1} \cdot (-o_i o_1) + \frac{y_2}{o_2} \cdot (-o_i o_2) + ... + \frac{y_i}{o_i} \cdot o_i (1 - o_i) + ... + \frac{y_k}{o_k} \cdot (-o_i o_k) ]$

$\displaystyle \frac{\partial L}{\partial h_i} = o_i \frac{y_1}{o_1} \cdot o_1 + o_i \frac{y_2}{o_2} \cdot  o_2 + ... + o_i \frac{y_i}{o_i} \cdot (o_i - 1) + ... + o_i \frac{y_k}{o_k} \cdot  o_k$

$\displaystyle \frac{\partial L}{\partial h_i} = o_i \frac{y_1}{o_1} \cdot o_1 + o_i \frac{y_2}{o_2} \cdot  o_2 + ... + (o_i)^2 \frac{y_i}{o_i} - o_i \frac{y_i}{o_i} + ... + o_i \frac{y_k}{o_k} \cdot  o_k$

$\displaystyle \frac{\partial L}{\partial h_i} = o_i y_1 + o_i y_2 + ... + o_i y_i - y_i + ... + o_i y_k$

$\displaystyle \frac{\partial L}{\partial h_i} = (o_i y_1 + o_i y_2 + ... + o_i y_i + ... + o_i y_k) - y_i$

$\displaystyle \frac{\partial L}{\partial h_i} = o_i\sum_{i=1}^{k}{y_i} - y_i$

$\displaystyle \frac{\partial L}{\partial h_i} = o_i - y_i \text{, dado que } \sum_{i=1}^{k}{y_i} = 1$